#### Python3 , gpu 설정
#### Install Pytorch

In [0]:
# Default Code
!pip3 install torch torchvision

# Code from Colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch
torch.__version__

'0.3.0.post4'

#### Install Google Drive SDK ( 암호 2번 인증 )

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmps8d3w9sv/pubring.gpg' created
gpg: /tmp/tmps8d3w9sv/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


#### Mount Google Drive

In [1]:
!ls
!mkdir -p drive
!ls
!google-drive-ocamlfuse drive
!ls drive

=  datalab  drive
=  datalab  drive
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
사진    2학년  a.jpg  d.jpg	   e.jpg       retrain.py
창고    3학년  b.jpg  eeg_graph.pb  f.jpg       test
1학년  4학년  c.jpg  eeg.ipynb	   predict.py  workspace


#### Retrain & Inception V3, 100000번 학습


In [4]:
!python3 drive/retrain.py --bottleneck_dir = ./drive/workspace/bottlenecks --model_dir=./drive/workspace/inception --output_graph=./drive/eeg_graph.pb --output_labels=./drive/workspace/eeg_labels.txt --image_dir ./drive/workspace/dataset --how_many_training_steps 100000

In [5]:
!python3 drive/predict.py drive/med/a.jpg
!python3 drive/predict.py drive/med/b.jpg
!python3 drive/predict.py drive/med/c.jpg
!python3 drive/predict.py drive/med/d.jpg
!python3 drive/predict.py drive/med/e.jpg
!python3 drive/predict.py drive/med/f.jpg

=== 예측 결과 ===
theta (1.21%)
smr (2.14%)
low beta (0.29%)
high beta (1.35%)
gamma (3.17%)
delta (2.39%)
alpha (89.44%)
=== 예측 결과 ===
theta (1.07%)
smr (2.13%)
low beta (0.11%)
high beta (0.94%)
gamma (4.40%)
delta (0.51%)
alpha (90.83%)
=== 예측 결과 ===
theta (0.11%)
smr (0.27%)
low beta (3.34%)
high beta (0.07%)
gamma (1.51%)
delta (1.97%)
alpha (92.74%)
=== 예측 결과 ===
theta (0.31%)
smr (3.30%)
low beta (1.41%)
high beta (0.29%)
gamma (0.76%)
delta (0.65%)
alpha (93.28%)
=== 예측 결과 ===
theta (2.08%)
smr (0.94%)
low beta (86.41%)
high beta (0.03%)
gamma (0.92%)
delta (2.85%)
alpha (6.78%)
=== 예측 결과 ===
theta (0.49%)
smr (0.35%)
low beta (1.33%)
high beta (0.08%)
gamma (0.06%)
delta (95.37%)
alpha (2.32%)


In [6]:
!python3 drive/strip_unused.py --input_graph="./drive/eeg_graph.pb" --output_graph="./drive/stripped_graph.pb" --input_node_names="Mul" --output_node_names="final_result" --input_binary=true

997 ops in the final graph.
